# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [101]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [102]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [103]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [104]:
ed = Website("https://www.stanford.edu")
ed.links

['#main-content',
 'https://www.stanford.edu/',
 'https://www.stanford.edu/student-gateway/',
 'https://www.stanford.edu/faculty-staff-gateway/',
 'http://parents.stanford.edu/',
 'https://visit.stanford.edu',
 'http://alumni.stanford.edu/',
 'https://www.stanford.edu/news/',
 'https://www.stanford.edu/events/',
 'https://www.stanford.edu/academics/',
 'https://www.stanford.edu/research/',
 'https://www.stanford.edu/health-care/',
 'https://www.stanford.edu/campus-life/',
 'https://www.stanford.edu/admission/',
 'https://www.stanford.edu/about/',
 'https://campus-map.stanford.edu/',
 'https://profiles.stanford.edu/',
 '#main-content',
 'https://www.stanford.edu/about/',
 'https://news.stanford.edu/stories/2025/01/assessing-wildfire-health-risks',
 'https://news.stanford.edu/stories/2025/01/assessing-wildfire-health-risks',
 'https://news.stanford.edu/stories/2025/01/ai-cancer-prognosis',
 'https://news.stanford.edu/stories/2025/01/ai-cancer-prognosis',
 'https://news.stanford.edu/stori

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [105]:
link_system_prompt = """
You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company.

Here are some examples of good link classifications:

Example 1 - A Technology Company:
Input links:
/about-us
/privacy
/careers
/contact
/products
/terms

Output:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about-us"},
        {"type": "careers page", "url": "https://company.com/careers"},
        {"type": "products page", "url": "https://company.com/products"}
    ]
}

Example 2 - A Consulting Firm:
Input links:
/who-we-are
/services
/join-our-team
/blog
/cookie-policy
/case-studies

Output:
{
    "links": [
        {"type": "company overview", "url": "https://firm.com/who-we-are"},
        {"type": "services page", "url": "https://firm.com/services"},
        {"type": "careers page", "url": "https://firm.com/join-our-team"},
        {"type": "case studies", "url": "https://firm.com/case-studies"}
    ]
}


Example 3 - An Educational Institution:
Input links:
/academics
/admissions
/research
/campus-life
/alumni
/contact

Output:
{
    "links": [
        {"type": "academic programs", "url": "https://university.edu/academics"},
        {"type": "admissions", "url": "https://university.edu/admissions"},
        {"type": "research", "url": "https://university.edu/research"},
        {"type": "campus life", "url": "https://university.edu/campus-life"}
    ]
}

Now analyze the following links and categorize them similarly, providing full URLs. 
Exclude any links to privacy policies, terms of service, or email addresses.
Respond in JSON format as shown in the examples above.
"""

link_system_prompt += """
Example of links to exclude:
- mailto:contact@company.com
- /privacy-policy
- /terms-of-service
- /cookies
- /sitemap.xml
"""

In [106]:
print(link_system_prompt)


You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company.

Here are some examples of good link classifications:

Example 1 - A Technology Company:
Input links:
/about-us
/privacy
/careers
/contact
/products
/terms

Output:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about-us"},
        {"type": "careers page", "url": "https://company.com/careers"},
        {"type": "products page", "url": "https://company.com/products"}
    ]
}

Example 2 - A Consulting Firm:
Input links:
/who-we-are
/services
/join-our-team
/blog
/cookie-policy
/case-studies

Output:
{
    "links": [
        {"type": "company overview", "url": "https://firm.com/who-we-are"},
        {"type": "services page", "url": "https://firm.com/services"},
        {"type": "careers page", "url": "https://firm.com/join-our-team"},
        {"type": "case studies", "url": "https://firm.com/ca

In [107]:
# def get_links_user_prompt(website):
#     user_prompt = f"Here is the list of links on the website of {website.url} - "
#     user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
# Do not include Terms of Service, Privacy, email links.\n"
#     user_prompt += "Links (some might be relative links):\n"
#     user_prompt += "\n".join(website.links)
#     return user_prompt


def get_links_user_prompt(website):
    base_url = website.url.rstrip('/')
    user_prompt = f"Website URL: {base_url}\n\n"
    user_prompt += "Found links:\n"
    
    # Convert relative links to absolute URLs
    formatted_links = []
    for link in website.links:
        if link and isinstance(link, str):  # Check if link is valid
            if link.startswith('/'):
                formatted_links.append(f"{base_url}{link}")
            elif link.startswith('http'):
                formatted_links.append(link)
            # Skip mailto: and other excluded links
            elif not any(excluded in link.lower() for excluded in ['mailto:', 'privacy', 'terms', 'cookies', 'sitemap']):
                # Handle other relative links
                formatted_links.append(f"{base_url}/{link.lstrip('/')}")
    
    user_prompt += "\n".join(formatted_links)
    return user_prompt

In [108]:
print(get_links_user_prompt(ed))

Website URL: https://www.stanford.edu

Found links:
https://www.stanford.edu/#main-content
https://www.stanford.edu/
https://www.stanford.edu/student-gateway/
https://www.stanford.edu/faculty-staff-gateway/
http://parents.stanford.edu/
https://visit.stanford.edu
http://alumni.stanford.edu/
https://www.stanford.edu/news/
https://www.stanford.edu/events/
https://www.stanford.edu/academics/
https://www.stanford.edu/research/
https://www.stanford.edu/health-care/
https://www.stanford.edu/campus-life/
https://www.stanford.edu/admission/
https://www.stanford.edu/about/
https://campus-map.stanford.edu/
https://profiles.stanford.edu/
https://www.stanford.edu/#main-content
https://www.stanford.edu/about/
https://news.stanford.edu/stories/2025/01/assessing-wildfire-health-risks
https://news.stanford.edu/stories/2025/01/assessing-wildfire-health-risks
https://news.stanford.edu/stories/2025/01/ai-cancer-prognosis
https://news.stanford.edu/stories/2025/01/ai-cancer-prognosis
https://news.stanford.e

In [109]:
# def get_links(url):
#     website = Website(url)
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)


def get_links(url, industry_type=None):
    try:
        website = Website(url)
        user_prompt = get_links_user_prompt(website)
        
        # Add industry context if provided
        if industry_type:
            user_prompt += f"\nThis is a {industry_type} company website."
        
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        result = response.choices[0].message.content
        return json.loads(result)
    except Exception as e:
        print(f"Error processing links: {str(e)}")
        return {"links": []}  # Return empty list in case of error


In [110]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://stanford.com")
huggingface.links

['#main-content',
 'https://www.stanford.edu/',
 'https://www.stanford.edu/student-gateway/',
 'https://www.stanford.edu/faculty-staff-gateway/',
 'http://parents.stanford.edu/',
 'https://visit.stanford.edu',
 'http://alumni.stanford.edu/',
 'https://www.stanford.edu/news/',
 'https://www.stanford.edu/events/',
 'https://www.stanford.edu/academics/',
 'https://www.stanford.edu/research/',
 'https://www.stanford.edu/health-care/',
 'https://www.stanford.edu/campus-life/',
 'https://www.stanford.edu/admission/',
 'https://www.stanford.edu/about/',
 'https://campus-map.stanford.edu/',
 'https://profiles.stanford.edu/',
 '#main-content',
 'https://www.stanford.edu/about/',
 'https://news.stanford.edu/stories/2025/01/assessing-wildfire-health-risks',
 'https://news.stanford.edu/stories/2025/01/assessing-wildfire-health-risks',
 'https://news.stanford.edu/stories/2025/01/ai-cancer-prognosis',
 'https://news.stanford.edu/stories/2025/01/ai-cancer-prognosis',
 'https://news.stanford.edu/stori

In [111]:
get_links("https://standord.com")

{'links': [{'type': 'homepage', 'url': 'https://standord.com/'},
  {'type': 'about page', 'url': 'https://standord.com/about'},
  {'type': 'publications page', 'url': 'https://standord.com/publications'},
  {'type': 'approach page', 'url': 'https://standord.com/approach'},
  {'type': 'contact page', 'url': 'https://standord.com/contact-us'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [112]:
# def get_all_details(url):
#     result = "Landing page:\n"
#     result += Website(url).get_contents()
#     links = get_links(url)
#     print("Found links:", links)
#     for link in links["links"]:
#         result += f"\n\n{link['type']}\n"
#         result += Website(link["url"]).get_contents()
#     return result


def get_all_details(url, industry_type=None):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url, industry_type)
    print("Found links:", json.dumps(links, indent=2))
    
    for link in links["links"]:
        try:
            result += f"\n\n{link['type']}\n"
            result += Website(link["url"]).get_contents()
        except Exception as e:
            print(f"Error processing link {link['url']}: {str(e)}")
            continue
    
    return result

In [113]:
print(get_all_details("https://stanford.com"))

Found links: {
  "links": [
    {
      "type": "university overview",
      "url": "https://www.stanford.edu/about/"
    },
    {
      "type": "academic programs",
      "url": "https://www.stanford.edu/academics/"
    },
    {
      "type": "admissions",
      "url": "https://www.stanford.edu/admission/"
    },
    {
      "type": "research",
      "url": "https://www.stanford.edu/research/"
    },
    {
      "type": "campus life",
      "url": "https://www.stanford.edu/campus-life/"
    },
    {
      "type": "health care",
      "url": "https://www.stanford.edu/health-care/"
    },
    {
      "type": "events",
      "url": "https://www.stanford.edu/events/"
    },
    {
      "type": "news",
      "url": "https://www.stanford.edu/news/"
    },
    {
      "type": "alumni",
      "url": "http://alumni.stanford.edu/"
    },
    {
      "type": "student resources",
      "url": "https://undergrad.stanford.edu/"
    }
  ]
}
Landing page:
Webpage Title:
Stanford University
Webpage Co

In [114]:
system_prompt = """You are an assistant that analyzes website content and creates a snarky brochure with specified sections.

FORMAT REQUIRED:
# [Company Name]: The Real Story

## What They Actually Do
[A snarky description of their main products/services]

## Company Culture & Vibes
[A witty take on their work environment and values]

## Why They Think They're Special
[A sarcastic overview of their unique selling points]

## The Fine Print
[Humorous disclaimers and caveats about working with/for them]

## Should You Join?
[A playful pros and cons list for potential employees]

RULES:
- Keep sections in the exact order specified above
- Use markdown formatting
- Maintain a snarky but not mean-spirited tone
- Include specific details from the website content
- Keep each section to 2-3 paragraphs maximum

Example Format:
# Acme Corp: The Real Story

## What They Actually Do
[Content here]

[Continue with other sections...]"""

system_prompt += """

SECTION REQUIREMENTS:
1. "What They Actually Do" must include:
   - Main product/service description
   - Target market
   - Key features

2. "Company Culture & Vibes" must include:
   - Office environment
   - Work-life balance mentions
   - Team structure

[Continue with other sections...]"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [115]:
# def get_brochure_user_prompt(company_name, url):
#     user_prompt = f"You are looking at a company called: {company_name}\n"
#     user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
#     user_prompt += get_all_details(url)
#     user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
#     return user_prompt


def get_brochure_user_prompt(company_name, url, industry_type=None):
    user_prompt = f"Create a brochure for {company_name}"
    if industry_type:
        user_prompt += f", a company in the {industry_type} industry"
    user_prompt += ". Use ONLY the following website content to create the brochure:\n\n"
    user_prompt += get_all_details(url, industry_type)
    user_prompt += "\nMake sure to follow the exact format specified in the system prompt."
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [116]:
get_brochure_user_prompt("Stanford", "https://stanford.com")

Found links: {
  "links": [
    {
      "type": "academic programs",
      "url": "https://www.stanford.edu/academics/"
    },
    {
      "type": "research",
      "url": "https://www.stanford.edu/research/"
    },
    {
      "type": "health care",
      "url": "https://www.stanford.edu/health-care/"
    },
    {
      "type": "campus life",
      "url": "https://www.stanford.edu/campus-life/"
    },
    {
      "type": "admissions",
      "url": "https://www.stanford.edu/admission/"
    },
    {
      "type": "about the university",
      "url": "https://www.stanford.edu/about/"
    },
    {
      "type": "events",
      "url": "https://www.stanford.edu/events/"
    },
    {
      "type": "news",
      "url": "https://www.stanford.edu/news/"
    },
    {
      "type": "undergraduate programs",
      "url": "https://undergrad.stanford.edu/"
    },
    {
      "type": "graduate admissions",
      "url": "https://gradadmissions.stanford.edu/explore-programs"
    },
    {
      "type": 

'Create a brochure for Stanford. Use ONLY the following website content to create the brochure:\n\nLanding page:\nWebpage Title:\nStanford University\nWebpage Contents:\nSkip to content\nStanford University\nInformation for:\nStudents\nFaculty & Staff\nFamilies\nVisitors\nAlumni\nSearch\nMenu\nNews\nEvents\nAcademics\nResearch\nHealth Care\nCampus Life\nAdmission\nAbout\nSearch term\nSearch Stanford:\nWeb\nPeople\nSearch Web\nOther ways to search:\nMap\nProfiles\nStanford\nExplore Stanford\nMain Content\nA Societal Mission\nStanford was founded almost 150 years ago on a bedrock of societal purpose. Our mission is to contribute to the world by educating students for lives of leadership and contribution with integrity; advancing fundamental knowledge and cultivating creativity; leading in pioneering research for effective clinical therapies; and accelerating solutions and amplifying their impact.\nMore about Stanford\nCampus News\nStories about people, research, and innovation across the

In [124]:
def translate_to_spanish(text):
    """Translates the generated brochure to Spanish"""
    translation_prompt = """You are a professional translator. Translate the following markdown-formatted brochure to Spanish. 
    Maintain all markdown formatting exactly as is. Keep section titles in their translated Spanish form.
    """
    
    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": translation_prompt},
                {"role": "user", "content": text}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Translation error: {str(e)}"

In [125]:
# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))


def create_brochure(company_name, url, industry_type=None, tone="snarky", translate=False):
    # Define available tones
    tones = {
        "snarky": "use a sarcastic and witty tone",
        "professional": "use a formal but engaging tone",
        "casual": "use a friendly and conversational tone",
        "enthusiastic": "use an excited and energetic tone"
    }
    
    # Base system prompt
    base_system_prompt = """You are an assistant that analyzes website content and creates a brochure with specified sections.

    FORMAT REQUIRED:
    # [Company Name]: The Real Story

    ## What They Actually Do
    [A description of their main products/services]

    ## Company Culture & Vibes
    [A take on their work environment and values]

    ## Why They Think They're Special
    [An overview of their unique selling points]

    ## The Fine Print
    [Details about working with/for them]

    ## Should You Join?
    [A pros and cons list for potential employees]

    RULES:
    - Keep sections in the exact order specified above
    - Use markdown formatting
    - Use the specified tone
    - Include specific details from the website content
    - Keep each section to 2-3 paragraphs maximum"""
    
    # Create the modified system prompt with the specified tone
    system_prompt = base_system_prompt.replace(
        "Use the specified tone",
        tones.get(tone, "use a sarcastic and witty tone")
    )
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, industry_type)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

    # If translation is requested, translate and display
    if translate:
        print("\nTraducción en Español (Spanish Translation):")
        spanish_version = translate_to_spanish(result)
        display(Markdown(spanish_version))
        
        return result, spanish_version
    
    return result


# Get both English and Spanish versions
brochure, spanish = create_brochure(
    "HuggingFace", 
    "https://huggingface.co", 
    "AI technology", 
    tone="professional",
    translate=True
)

# Just English version
english_only = create_brochure(
    "HuggingFace", 
    "https://huggingface.co", 
    "AI technology", 
    tone="professional"
)



Found links: {
  "links": [
    {
      "type": "homepage",
      "url": "https://huggingface.co/"
    },
    {
      "type": "models page",
      "url": "https://huggingface.co/models"
    },
    {
      "type": "datasets page",
      "url": "https://huggingface.co/datasets"
    },
    {
      "type": "spaces page",
      "url": "https://huggingface.co/spaces"
    },
    {
      "type": "posts page",
      "url": "https://huggingface.co/posts"
    },
    {
      "type": "documentation",
      "url": "https://huggingface.co/docs"
    },
    {
      "type": "enterprise solutions",
      "url": "https://huggingface.co/enterprise"
    },
    {
      "type": "pricing page",
      "url": "https://huggingface.co/pricing"
    },
    {
      "type": "blog",
      "url": "https://huggingface.co/blog"
    },
    {
      "type": "learn page",
      "url": "https://huggingface.co/learn"
    }
  ]
}


# HuggingFace: The Real Story

## What They Actually Do
Hugging Face is a pioneering platform designed to facilitate collaboration within the machine learning (ML) community. At its core, Hugging Face provides a comprehensive hub for users to discover, create, and collaborate on models, datasets, and applications. With an extensive library of over 400,000 models and 100,000 datasets, the platform serves as a vital resource for both budding and advanced practitioners in the field of AI. Additionally, Hugging Face offers specialized solutions, such as Compute and Enterprise plans, catering to the rigorous demands of organizations seeking to leverage cutting-edge AI technologies.

The platform also includes dynamic features such as "Spaces," allowing users to run and showcase their machine learning applications easily. Hugging Face actively embraces open source principles, providing powerful tools like Transformers and Diffusers, which are integral to various ML tasks, from text generation to image analysis.

## Company Culture & Vibes
At Hugging Face, the company mission revolves around building a welcoming and collaborative community that transcends traditional corporate silos. The work environment emphasizes inclusivity, valuing diverse perspectives that contribute to innovation in AI. Employees are encouraged to be part of a collective effort that shapes the future of machine learning, fostering an atmosphere where experimentation and creativity can thrive.

The company values open-source principles and provides its employees with opportunities to engage with a global community of developers and researchers. This culture of transparency and knowledge sharing not only enhances personal growth but also drives the organization’s mission forward, making Hugging Face a vibrant place to cultivate one’s career in AI.

## Why They Think They're Special
Hugging Face distinguishes itself through its commitment to open source development and community collaboration. Its extensive catalog of models and datasets positions it as a go-to resource in the AI industry, catering to a diverse range of applications across different modalities, including text, audio, and visual data. The versatility of the platform ensures that users—from individual developers to large enterprises—can accelerate their machine learning projects without barriers.

Moreover, Hugging Face provides enterprise-grade solutions that combine advanced security with dedicated support, making it an ideal partner for organizations. With over 50,000 companies utilizing its services, Hugging Face has established itself as a leader in AI technology, inspiring confidence and driving adoption across industries.

## The Fine Print
Hugging Face operates with a clear focus on delivering value to its users while maintaining high ethical standards in AI development. The platform provides multiple pricing tiers, starting as low as $0.60 per hour for GPU computing, allowing users to choose a model that best fits their needs. For enterprises, competitive plans beginning at $20 per user per month include features like Single Sign-On, priority support, and access controls, ensuring a tailored approach to each organization's requirements.

Hugging Face also maintains comprehensive documentation and resources for users, supporting their journey from novice to expert. The transparent user agreements and terms of service further protect the expectations of both contributors and users of the platform.

## Should You Join?
### Pros
- **Innovative Environment**: Work at the forefront of AI and be part of a community that values creativity and collaboration.
- **Extensive Resources**: Have access to a plethora of models, datasets, and tools that can significantly enhance your work.
- **Career Growth**: Leverage opportunities for personal development and expand your professional network within an open-source ecosystem.

### Cons
- **Fast-Paced**: The rapidly evolving nature of AI technologies may require continual learning and adaptation.
- **Open-Source Challenges**: Contributing to open-source projects can sometimes mean navigating complex collaboration dynamics.
- **High Expectations**: Working with leading organizations may result in a demanding work environment with high standards for performance. 

Overall, if you are passionate about AI and eager to collaborate with like-minded individuals, Hugging Face may be the perfect place for you to thrive.


Traducción en Español (Spanish Translation):


# HuggingFace: La Verdadera Historia

## Lo Que Realmente Hacen
Hugging Face es una plataforma pionera diseñada para facilitar la colaboración dentro de la comunidad de aprendizaje automático (ML). En su núcleo, Hugging Face proporciona un centro integral para que los usuarios descubran, creen y colaboren en modelos, conjuntos de datos y aplicaciones. Con una extensa biblioteca de más de 400,000 modelos y 100,000 conjuntos de datos, la plataforma sirve como un recurso vital tanto para practicantes novatos como avanzados en el campo de la IA. Además, Hugging Face ofrece soluciones especializadas, como planes de Compute y Enterprise, que satisfacen las rigurosas demandas de organizaciones que buscan aprovechar tecnologías de IA de vanguardia.

La plataforma también incluye características dinámicas como “Espacios”, que permiten a los usuarios ejecutar y demostrar fácilmente sus aplicaciones de aprendizaje automático. Hugging Face adopta activamente principios de código abierto, proporcionando herramientas poderosas como Transformers y Diffusers, que son fundamentales para diversas tareas de ML, desde la generación de texto hasta el análisis de imágenes.

## Cultura Empresarial y Atmosfera
En Hugging Face, la misión de la empresa gira en torno a construir una comunidad acogedora y colaborativa que trascienda los silos corporativos tradicionales. El ambiente de trabajo enfatiza la inclusividad, valorando diversas perspectivas que contribuyen a la innovación en IA. Se anima a los empleados a ser parte de un esfuerzo colectivo que moldea el futuro del aprendizaje automático, fomentando una atmósfera donde la experimentación y la creatividad pueden prosperar.

La empresa valora los principios de código abierto y brinda a sus empleados oportunidades para interactuar con una comunidad global de desarrolladores e investigadores. Esta cultura de transparencia y compartición de conocimientos no solo mejora el crecimiento personal, sino que también impulsa la misión de la organización hacia adelante, haciendo de Hugging Face un lugar vibrante para cultivar la carrera de uno en IA.

## Por Qué Piensan Que Son Especiales
Hugging Face se distingue por su compromiso con el desarrollo de código abierto y la colaboración comunitaria. Su extenso catálogo de modelos y conjuntos de datos lo posiciona como un recurso imprescindible en la industria de IA, atendiendo una amplia gama de aplicaciones en diferentes modalidades, incluyendo texto, audio y datos visuales. La versatilidad de la plataforma asegura que los usuarios, desde desarrolladores individuales hasta grandes empresas, puedan acelerar sus proyectos de aprendizaje automático sin barreras.

Además, Hugging Face proporciona soluciones de nivel empresarial que combinan seguridad avanzada con soporte dedicado, convirtiéndolo en un socio ideal para las organizaciones. Con más de 50,000 empresas utilizando sus servicios, Hugging Face se ha establecido como un líder en tecnología de IA, inspirando confianza y fomentando la adopción en diversas industrias.

## La Letra Pequeña
Hugging Face opera con un enfoque claro en ofrecer valor a sus usuarios mientras mantiene altos estándares éticos en el desarrollo de IA. La plataforma ofrece múltiples niveles de precios, comenzando desde tan solo $0.60 por hora para computación GPU, permitiendo a los usuarios elegir un modelo que se adapte mejor a sus necesidades. Para las empresas, los planes competitivos que comienzan en $20 por usuario al mes incluyen características como Inicio de Sesión Único, soporte prioritario y controles de acceso, garantizando un enfoque a medida para los requisitos de cada organización.

Hugging Face también mantiene documentación exhaustiva y recursos para los usuarios, apoyando su viaje de novato a experto. Los acuerdos de usuario transparentes y los términos de servicio protegen aún más las expectativas tanto de los contribuyentes como de los usuarios de la plataforma.

## ¿Deberías Unirte?
### Ventajas
- **Ambiente Innovador**: Trabaja en la vanguardia de la IA y sé parte de una comunidad que valora la creatividad y la colaboración.
- **Recursos Extensos**: Ten acceso a una plétora de modelos, conjuntos de datos y herramientas que pueden mejorar significativamente tu trabajo.
- **Crecimiento Profesional**: Aprovecha las oportunidades de desarrollo personal y expande tu red profesional dentro de un ecosistema de código abierto.

### Desventajas
- **Rápido Desarrollo**: La naturaleza en rápida evolución de las tecnologías de IA puede requerir un aprendizaje y adaptación continuos.
- **Desafíos de Código Abierto**: Contribuir a proyectos de código abierto a veces puede significar navegar dinámicas de colaboración complejas.
- **Altas Expectativas**: Trabajar con organizaciones líderes puede resultar en un ambiente de trabajo exigente con altos estándares de desempeño.

En general, si eres un apasionado de la IA y estás ansioso por colaborar con individuos que piensan de manera similar, Hugging Face puede ser el lugar perfecto para que prosperes.

Found links: {
  "links": [
    {
      "type": "homepage",
      "url": "https://huggingface.co/"
    },
    {
      "type": "models page",
      "url": "https://huggingface.co/models"
    },
    {
      "type": "datasets page",
      "url": "https://huggingface.co/datasets"
    },
    {
      "type": "spaces page",
      "url": "https://huggingface.co/spaces"
    },
    {
      "type": "blog",
      "url": "https://huggingface.co/blog"
    },
    {
      "type": "enterprise page",
      "url": "https://huggingface.co/enterprise"
    },
    {
      "type": "pricing page",
      "url": "https://huggingface.co/pricing"
    },
    {
      "type": "documentation",
      "url": "https://huggingface.co/docs"
    },
    {
      "type": "join page",
      "url": "https://huggingface.co/join"
    },
    {
      "type": "learning resources",
      "url": "https://huggingface.co/learn"
    },
    {
      "type": "AI Tasks",
      "url": "https://huggingface.co/tasks"
    },
    {
      "type": "

# Hugging Face: The Real Story

## What They Actually Do
Hugging Face is at the forefront of the artificial intelligence revolution, serving as a collaborative platform where the machine learning community can access, share, and enhance models and datasets. They offer an extensive repository of over 400,000 models and 100,000 datasets, encompassing various modalities including text, image, video, audio, and even 3D. Hugging Face enables users to create, discover, and collaborate on machine learning applications efficiently while providing tools such as the HF Open Source stack and paid Compute solutions to accelerate the development process.

The company also promotes innovative libraries and tools, such as Transformers, Diffusers, and Tokenizers, which are essential for both research and production in state-of-the-art machine learning applications. By hosting unlimited public models and enabling access to advanced enterprise-grade solutions, Hugging Face is committed to building the future of AI technology collaboratively.

## Company Culture & Vibes
Hugging Face fosters a vibrant and inclusive culture heavily rooted in community collaboration and open-source principles. They are dedicated to breaking down barriers in AI development, making cutting-edge technology accessible to everyone. Hugging Face not only encourages creativity and innovation among its team members but also emphasizes the importance of support and shared learning within the broader machine learning community.

The environment at Hugging Face is characterized by a strong spirit of collaboration; individuals work together to solve challenges and celebrate each other’s successes. Upholding values centered around transparency and community engagement, employees are often motivated by a shared vision of empowering the AI community to build responsibly and effectively.

## Why They Think They're Special
Hugging Face distinguishes itself by being the home of a growing AI community that thrives on collaboration. With a platform that hosts more than 50,000 organizations, including major players like Amazon, Microsoft, and Google, they have created a powerful ecosystem that supports collective growth and innovation in AI. Their commitment to open-source solutions further ensures that advancements in machine learning are a collective effort rather than the domain of a few.

Moreover, their pioneering integration of state-of-the-art models, comprehensive datasets, and user-friendly deployment options creates a unique offering that stands out in the rapidly evolving AI landscape. Hugging Face’s focus on community building, alongside their robust enterprise solutions, positions them as a leader in the AI technology industry.

## The Fine Print
Working with or for Hugging Face means immersing yourself in a mission-driven environment where innovation and collaboration form the backbone of operations. Employees enjoy access to extensive resources, including advanced tools for machine learning development, while also engaging in continuous learning opportunities. The company prides itself on maintaining a supportive and inclusive atmosphere, with a promise of growth both individually and collectively.

Hugging Face also provides enterprise solutions that include dedicated support, resource groups, and advanced security measures, making it an attractive choice for organizations seeking to secure their AI environments. Their scalable platform is tailored for teams of all sizes, ensuring that users can effectively build and deploy AI solutions per their specific needs.

## Should You Join?
### Pros:
- **Collaborative Culture:** Work in an environment that values teamwork and community knowledge-sharing.
- **Leading Technology:** Access to cutting-edge tools and resources, ensuring you stay ahead in AI development.
- **Diverse Opportunities:** Engage with a broad range of projects across different modalities and industries.
- **Mission-Driven:** Contribute to a growing initiative that aims to democratize AI technology.

### Cons:
- **Pace of Change:** The rapid evolution of the technology can be overwhelming for some.
- **Resource Allocation:** With a focus on community support, some projects may require more time for completion.
- **Complexity in Collaboration:** Navigating collaborative projects can sometimes introduce challenges in alignment and communication.

In conclusion, joining Hugging Face means becoming part of a forward-thinking community that is shaping the future of AI, while offering a unique blend of challenges and rewards for its members.

In [126]:
create_brochure("Stanford", "https://stanford.co", "AI technology", tone="professional")


Found links: {
  "links": []
}


# Stanford: The Real Story

## What They Actually Do
Stanford operates within the AI technology industry, specializing in providing expert guidance and leadership in cybersecurity. Led by Jon Stanford, the company offers interim, fractional, and virtual security leadership to a diverse range of organizations, including enterprise, mid-market, venture capital, and private equity firms. Additionally, as the CEO and Managing Principal of Stanford Cyber Group LLC, they also engage with early-stage technology companies, focusing on security preparedness and ROI.

In a rapidly evolving security landscape filled with complexities, Stanford aids clients in navigating through these challenges. They emphasize emerging AI security, governance, and regulatory requirements, ensuring that businesses can implement effective security measures while aligning with industry standards.

## Company Culture & Vibes
At Stanford, the culture is driven by a commitment to understanding and addressing the unique security challenges faced by their clients. The work environment fosters collaboration between seasoned professionals who leverage their extensive expertise to provide tailored solutions. This culture of partnership ensures that every client receives personalized attention and strategic insights that are essential for navigating critical security decisions.

The company values transparency and open communication, both within the team and with clients. This commitment to dialogues promotes a sense of trust and accountability, enabling Stanford to build lasting relationships with their numerous stakeholders.

## Why They Think They're Special
Stanford distinguishes itself through its unparalleled expertise in cybersecurity and its deep understanding of the complexities surrounding emerging AI technologies. They pride themselves on not just being advisors but partners who actively engage with their clients to develop clear strategies that lead to critical outcomes. Their global experience across various sectors, including financial services, retail, and entertainment, equips them to handle diverse security needs.

The emphasis on a fractional and virtual leadership model allows organizations to engage with high-level security expertise without the need for full-time hires, making effective security management more accessible for businesses of all sizes.

## The Fine Print
Working with Stanford means gaining access to a suite of specialized services designed to enhance your organization's security posture. Clients can expect a strategic approach that addresses not just immediate security concerns but also looks at long-term goals and investments. The firm focuses on clear communication and tailored solutions, ensuring that each engagement is aligned with the clients' specific needs.

Stanford's involvement spans across various industries, allowing them to bring a wealth of knowledge to every project. Their focus on both security preparedness and ROI means that they strive to deliver measurable outcomes that positively impact the bottom line.

## Should You Join?
### Pros:
- **Expert Leadership**: Gain insights and guidance from seasoned professionals with extensive experience in cybersecurity.
- **Flexible Engagement**: Opportunity to work in a model that supports fractional and virtual leadership roles, allowing for diverse and dynamic project engagements.
- **Broad Industry Exposure**: Work with clients across various sectors, enhancing your experience and skills in different environments.

### Cons:
- **High Expectations**: The complexity of client needs means that the pressure to deliver results can be significant.
- **Dynamic Environment**: Rapid changes in technology and cybersecurity regulations require constant learning and adaptation.
- **Potentially Irregular Workload**: Depending on client demands, project work may lead to variable workloads, which may not provide the consistency some professionals prefer.

"# Stanford: The Real Story\n\n## What They Actually Do\nStanford operates within the AI technology industry, specializing in providing expert guidance and leadership in cybersecurity. Led by Jon Stanford, the company offers interim, fractional, and virtual security leadership to a diverse range of organizations, including enterprise, mid-market, venture capital, and private equity firms. Additionally, as the CEO and Managing Principal of Stanford Cyber Group LLC, they also engage with early-stage technology companies, focusing on security preparedness and ROI.\n\nIn a rapidly evolving security landscape filled with complexities, Stanford aids clients in navigating through these challenges. They emphasize emerging AI security, governance, and regulatory requirements, ensuring that businesses can implement effective security measures while aligning with industry standards.\n\n## Company Culture & Vibes\nAt Stanford, the culture is driven by a commitment to understanding and addressing t

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [72]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [93]:
stream_brochure("Stanford", "https://stanford.com")

Found links: {
  "links": [
    {
      "type": "homepage",
      "url": "https://stanford.com/"
    },
    {
      "type": "academic programs",
      "url": "https://www.stanford.edu/academics/"
    },
    {
      "type": "research",
      "url": "https://www.stanford.edu/research/"
    },
    {
      "type": "campus life",
      "url": "https://www.stanford.edu/campus-life/"
    },
    {
      "type": "admissions",
      "url": "https://www.stanford.edu/admission/"
    },
    {
      "type": "about page",
      "url": "https://www.stanford.edu/about/"
    },
    {
      "type": "events",
      "url": "https://www.stanford.edu/events/"
    },
    {
      "type": "news",
      "url": "https://www.stanford.edu/news/"
    },
    {
      "type": "health care",
      "url": "https://www.stanford.edu/health-care/"
    },
    {
      "type": "student services",
      "url": "https://studentaffairs.stanford.edu/"
    },
    {
      "type": "alumni",
      "url": "http://alumni.stanford.edu/"


# Welcome to Stanford University: Your Gateway to Overachieving!

## Academics: Because Mediocrity is Not an Option
At Stanford, we’ve decided that our students shouldn’t just take a class or two. No, they should aim for nothing less than to “engage as leaders in a complex world.” It’s quite the overachieving agenda, but who wouldn’t want to be thrust into a whirlwind of academic excellence? With seven schools crammed full of fancy degrees, you’re bound to find something that not only fills your brain but puts your future on a silver platter!

## Campus Life: The Royal Treatment
If you thought living in a campus dorm was synonymous with sharing a cramped space and cold pizza, think again! At Stanford, you can enjoy diverse housing options, extraordinary dining (because the cafeteria food definitely isn’t the typical fare), and a whopping **600 student organizations**, because what’s life without endless extracurricular commitments? Who needs free time anyway?

## Health Care: Leading the Revolution (And the Waiting Room)
We’re revolutionizing precision health over here, one groundbreaking AI discovery at a time. Whether you’re seeking treatment for a mild headache or trying to outsmart Type 2 diabetes, we've got you covered—just don't mind the 10-hour wait for an appointment. After all, innovation takes time, right?

## Research: For the Curious Overachiever
Did we mention our research budget is a cool **$1.98 billion**? That means we’re throwing money at groundbreaking research without looking back. Want to study the effects of climate change during your lunch break? Go ahead! With **20 libraries** holding over **12 million items**, you’ll have all the resources you need to deeply ponder life’s most pressing questions... after discovering what’s for lunch.

## Career Opportunities: The Elite Training Ground
Looking for a job? Why not join the ranks of the academic elite? Stanford isn’t just about enrolling in courses; it’s about embarking on a life of continuous learning with truly grave expectations. We seek recruits who possess an “abundance of energy and curiosity”—because, apparently, lounging in your pajamas watching Netflix won’t cut it.

## Customer Experience: Seriously, Who's In Charge Here?
At Stanford, we have a ton of different "customers." You’ve got students, alumni, the families of students (gosh, we hope they don't need to catch their breath), and faculty making sure everything runs smoothly. You might think it’s chaotic, but it’s just Stanford's way of showing how seriously we take excellence in education. 

---

### In Conclusion:
Stanford University - where dreams don’t just die; they’re meticulously refined to become the next big thing in the world. So, are you ready to overcommit your life to endless possibilities and relentless striving for excellence? Of course, you are! Welcome to the Farm.

In [94]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Stanford", "https://stanford.com")

Found links: {
  "links": [
    {
      "type": "academics page",
      "url": "https://www.stanford.edu/academics/"
    },
    {
      "type": "research page",
      "url": "https://www.stanford.edu/research/"
    },
    {
      "type": "campus life page",
      "url": "https://www.stanford.edu/campus-life/"
    },
    {
      "type": "admissions page",
      "url": "https://www.stanford.edu/admission/"
    },
    {
      "type": "about page",
      "url": "https://www.stanford.edu/about/"
    },
    {
      "type": "health care page",
      "url": "https://www.stanford.edu/health-care/"
    },
    {
      "type": "events page",
      "url": "https://www.stanford.edu/events/"
    },
    {
      "type": "life-long learning page",
      "url": "https://lifelonglearning.stanford.edu/"
    },
    {
      "type": "student gateway",
      "url": "https://www.stanford.edu/student-gateway/"
    },
    {
      "type": "faculty and staff gateway",
      "url": "https://www.stanford.edu/faculty-

# Welcome to Stanford: The Exclusive Club of Overachievers

### Mission: World Domination (or Something Like That) 
Stanford was plopped into existence nearly 150 years ago with the grand mission of changing the world. Yes, while you were contemplating your cereal choices this morning, they were busy plotting to "educate students for lives of leadership and contribution with integrity." Because who doesn’t want their kids to grow up to be absolute pillars of society, right?

---

### "Academics”: The Only Place You Can Throw Money and Time 
Sure, they offer a rich learning experience that you probably can’t find anywhere else. Seven whole schools to choose from: *Medicine, Law, Humanities & Sciences, Engineering, Education, Sustainability,* and *Business.* Imagine having to narrow down your future to just one of those paths. No pressure!

---

### Research: Sponsored Budget of Nearly Two Billion Dollars 
That’s right, they've got a whopping $1.98 billion in sponsored research funding. Just think! With that kind of cash, you could fund a small country or, you know, an AI model that predicts your demise after stressing over deadlines.

---

### Customers or... You Mean Students? 
Here, you'll find a kaleidoscope of diverse housing, *over 600 student organizations,* and exceptional dining (because who needs to eat ramen when you can have artisanal avocado toast?). Are you ready to live the dream?

---

### Careers: Join the Elite Ranks or Don't 
Why join Stanford, you ask? Besides the whole "changing the world" thing, you also have the opportunity to work alongside some of the brightest minds on the planet. It's like being invited to a dinner party where everyone else is a Nobel laureate and you just enjoy the food. 

---

### Stanford's Campus Life: An Instagrammer's Paradise Personified
Vibrant, colorful and oh-so-inventive; Environmentally friendly? Check. With state-of-the-art facilities and a rich tradition of creativity, you’ll blend in like a model on a magazine cover. And don’t even get us started on their athletics program. A staggering 136 NCAA championships might just give you an exercise-induced meltdown. 

---

### The Conclusion: 
If you've got the ambition of a thousand suns and the desperation to succeed, Stanford is basically waiting for you with open arms. Just remember, if you can’t contribute to a better world and you're contemplating your life choices, there’s always the chance to grab a latte and try again. 

*Join Stanford University where the competition is fierce, the research is boundless, and the dining halls serve food better than your last three dates!*  🚀

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>